In [ ]:
import pandas as pd
data=pd.read_csv('/content/news_filtered_2024_p1.csv')
data[['url','cc_url','content']]

In [75]:
available_models = client.models.list()
print(available_models)


ModelListResponse(data=[Model(id='llama-3.3-70b-versatile', created=1733447754, object='model', owned_by='Meta', active=True, context_window=131072, public_apps=None, max_completion_tokens=32768), Model(id='whisper-large-v3-turbo', created=1728413088, object='model', owned_by='OpenAI', active=True, context_window=448, public_apps=None, max_completion_tokens=448), Model(id='groq/compound-mini', created=1756949707, object='model', owned_by='Groq', active=True, context_window=131072, public_apps=None, max_completion_tokens=8192), Model(id='meta-llama/llama-prompt-guard-2-22m', created=1748632101, object='model', owned_by='Meta', active=True, context_window=512, public_apps=None, max_completion_tokens=512), Model(id='meta-llama/llama-guard-4-12b', created=1746743847, object='model', owned_by='Meta', active=True, context_window=131072, public_apps=None, max_completion_tokens=1024), Model(id='meta-llama/llama-4-maverick-17b-128e-instruct', created=1743877158, object='model', owned_by='Meta',

In [79]:
import pandas as pd
import csv
from groq import Groq
from io import StringIO
client = Groq(api_key="")


# Prompt that asks the LLM to return only one instrument per news item
boilerplate = """
You will be given a government news item.

Instructions:
1. based on the text mentioned in the news item classify it into one of the following main categories:
   - Welfare (e.g., grants, benefits, subsidies)
   - Monetary (e.g., tax changes, fiscal incentives)
   - Legislative (e.g., new laws, regulations, statutory changes)
   - Administrative (e.g., new guidelines, standards, reporting requirements)
   - Institutional (e.g., creation of new agencies)
   - Market-based (e.g., trading permits, carbon pricing)
   - Informational (e.g., public awareness campaigns, nudging)
   - Other (e.g., strategic plans, consultations)
2. If relevant, add a secondary classification (e.g., both welfare and legislative).
Output format instructions:
3. then from the news item identify one explicit or implicit policy instruments mentioned in the text based on which class is identified.

- Return a strict CSV table with exactly these columns:
  "Main Classification","Secondary Classification",  "Policy Instrument"
- Only one row per news item
- Each column must be quoted with double quotes
- Do NOT include any other text or notes
- Even if the value contains commas, keep it in its own column

"""

rows_for_final_df = []

for i in range(10):
    news_text = data['content'][i]
    url = data['url'][i]
    cc_url = data['cc_url'][i] if 'cc_url' in data else None

    prompt = boilerplate + "\n" + news_text

    response = client.chat.completions.create(
        model="meta-llama/llama-prompt-guard-2-22m",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )

    csv_text = response.choices[0].message.content.strip()

    # Parse CSV safely
    f = StringIO(csv_text)
    reader = csv.reader(f, quotechar='"')

    try:
        next(reader)  # skip header
        first_row = next(reader)
    except StopIteration:
        # Fallback if LLM returned nothing
        first_row = ["None", "Other", ""]

    # Ensure all columns are present
    while len(first_row) < 3:
        first_row.append("")

    rows_for_final_df.append({
        "Main Classification": first_row[0],
        "Secondary Classification": first_row[1],
        "Policy Instrument": first_row[2],
        "url": url,
        "cc_url": cc_url
    })

# Create DataFrame
final_df = pd.DataFrame(rows_for_final_df)
print(final_df)


BadRequestError: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages'}}

In [ ]:
final_df.insert(0, "id", [f"25_{i}" for i in range(len(final_df))])
print(final_df)
import csv

# Save final_df to CSV with all values quoted
final_df.to_csv("classified_news_filtered_2024_p1.csv", index=False, quoting=csv.QUOTE_ALL)
